In [1]:
!pip install sentence-transformers datasets huggingface_hub faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 67.5 MB/s eta 0:00:00


In [2]:
# 실습 데이터를 내려받고 예시 데이터 확인
from datasets import load_dataset
klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_train[0]

README.md: 0.00B [00:00, ?B/s]

mrc/train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

mrc/validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [ ]:
# 기본 임베딩 모델 불러오기
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('Laseung/klue-roberta-base-klue-sts')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

In [ ]:
#데이터 전처리
from datasets import load_dataset
klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_test = load_dataset('klue', 'mrc', split='validation')

df_train = klue_mrc_train.to_pandas()
df_test = klue_mrc_test.to_pandas()

df_train = df_train[['title', 'question', 'context']]
df_test = df_test[['title', 'question', 'context']]

In [ ]:
# 질문과 관련이 없는 기사를 irrelevant context 컬럼에 추가
def add_ir_context(df):
    irrelevant_contexts = []
    for idx, row in df.iterrows():
        title = row['title']
        irrelevant_contexts.append(df.query(f"title != '{title}'").sample(n=1)['context'].values[0])
    df['irrelevant_context'] = irrelevant_contexts
    return df

df_train_ir = add_ir_context(df_train)
df_test_ir = add_ir_context(df_test)

In [ ]:
# 성능 평가에 사용할 데이터 생성
from sentence_transformers import InputExample

examples = []
for idx, row in df_test_ir.iterrows():
    examples.append(InputExample(texts=[row['question'], row['context']], label=1))
    examples.append(InputExample(texts=[row['question'], row['irrelevant_context']], label=0))


In [ ]:
#기본 임베딩 모델의 성능 평가 결과 확인
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(examples)

evaluator(sentence_model)

In [ ]:
# 긍정 데이터만으로 학습 데이터 구성
train_samples = []
for idx, row in df_train_ir.iterrows():
    train_samples.append(InputExample(texts=[row['question'], row['context']]))

In [ ]:
# 중복 학습 데이터 제거
from sentence_transformers import datasets

batch_size = 16

loader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=batch_size)

In [ ]:
# MNR 손실 함수 불러오기
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(sentence_model)


In [ ]:
# MRC 데이터셋으로 미세 조정
epochs = 1
save_path = 'output/klue_mrc_mnr'

sentence_model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmpu_steps=100,
    output_path=save_path,
    show_progress_bar=True
    )

In [ ]:
# 미세 조정한 모델 성능 평가 확인
evaluator(sentence_model)

In [ ]:
# 허깅페이스 허브에 미세 조정한 모델 업로드
from huggingface_hub import HfApi
from huggingface_hub import login

login(token='본인의 허깅 페이스 KEY 값')

api = HfApi()
repo_id = "klue-roberta-base-klue-sts-mrc-mnr-finetuned"
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=save_path,
    repo_id=f"Laseung/{repo_id}",
    repo_type="model",
)